<a href="https://colab.research.google.com/github/davidye007/Particle_Collision_Research/blob/main/shred_with_brownian_collisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### install packages
%%capture
### changes
!pip install opencv-python
!pip install seaborn
!pip install scipy
!pip install process-data
!pip install pytorch_forecasting

In [ ]:
### import packages
import numpy as np
import pytorch_forecasting
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import random
import cv2
import numpy as np
from scipy import stats
import seaborn as sns
import seaborn as sns
import pandas as pd
from dataclasses import make_dataclass
import pdb
from processdata import load_data
from processdata import TimeSeriesDataset
import models
import torch
import scipy.io
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# fetch video
vid = cv2.VideoCapture("Granular Brownian Motion Square.mp4")
# attempt to extract first frame
success, frame = vid.read()
# create dataframe for storing particle center coordinates and frame number
df = pd.DataFrame(columns = ['x','y','r','frame'])
# set counter to frame zero
counter = 0;
# max frame count is 10
max = 100000
while (success and counter < max):
  # convert frame to greyscale
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  # Blur using 3 * 3 kernel
  gray_blurred = cv2.blur(gray, (3, 3))
  # Apply Hough transform on blurred frame
  detected_circles = cv2.HoughCircles(gray_blurred,cv2.HOUGH_GRADIENT, dp = 1, minDist = 8, param1 = 50,
                                    param2 = 9, minRadius = 3, maxRadius = 5)
  # Check if circle detected
  if detected_circles is not None:
    # Round circle parameters x, y and r to integers
    detected_circles = np.uint16(np.around(detected_circles))
    for circle in detected_circles[0, :]:
       x, y, r = circle[0], circle[1], circle[2]
       df.loc[df.shape[0]] = [x, y, r, counter] 
       # draw circumfrence of detected circle
       # cv2.circle(frame, (np.uint16(np.around(x)), np.uint16(np.around(y))), r, (255, 0, 255), 2)
       # draw center of detected circle (for verification)
       cv2.circle(frame, (x, y), radius = 1, color = (0, 0, 255), thickness = 2)
  else:
    print("no circles detected")
  # mark boundaries for random sensor location placewment with green box
  # xlim (10,690) and ylim (10,710)
  cv2.line(frame, (10, 10), (690, 10), color = (0,255,0), thickness = 1)
  cv2.line(frame, (690, 10), (690, 710), color = (0,255,0), thickness = 1)
  cv2.line(frame, (10, 10), (10, 710), color = (0,255,0), thickness = 1)
  cv2.line(frame, (10, 710), (690, 710), color = (0,255,0), thickness = 1)
  # show detected particles (for verification)
  # cv2_imshow(frame)
  # select next frame
  success, frame = vid.read()
  # update counter
  counter = counter + 1
  if (counter == max):
      print('reached max frames')

In [ ]:
from tempfile import TemporaryFile
detected = TemporaryFile()
np.save(detected, df)

In [ ]:
# set seed for reproducibility
random.seed(20)
# number of sensors
num_sensors = 3
# randomly select three sensor locations
sensor_locations = [(random.randint(10, 690), random.randint(10, 710)) for i in range(num_sensors)]
# matrix Q of densities: shape(num of frames, num of sensors)
Q = np.zeros((counter,num_sensors))
for i in range(counter):
  single_frame = df[df["frame"] == i]
  x = single_frame["x"].astype(float)
  y = single_frame["y"].astype(float)
  # (row: x,y|column: every ball)
  values = np.vstack((x.ravel(), y.ravel()))
  kernel = stats.gaussian_kde(values, bw_method=0.3)
  for j in range(num_sensors):
    Q[i,j] = kernel.pdf(sensor_locations[j])

In [ ]:
# set seed for reproducibility
random.seed(20)
num_sensors = 3 
lags = 52
load_X = Q
# timesteps: n
n = load_X.shape[0]
# sensors: m
m = load_X.shape[1]
# train indicies column vector: choose 1000 timesteps from (total_timesteps_n - lags) timesteps
train_indices = np.random.choice(n - lags, size=1000, replace=False)
# column vector of ones with size (total_timesteps_n - lags)
mask = np.ones(n - lags)
# change elements corresponding to train_indicies to zero
mask[train_indices] = 0
# validation and test set: mask indicies that are non-zero
valid_test_indices = np.arange(0, n - lags)[np.where(mask!=0)[0]]
# 0,2,4,6... end
valid_indices = valid_test_indices[::2]
# 1,3,5,7... end
test_indices = valid_test_indices[1::2]
### tansform all data
sc = MinMaxScaler()
sc = sc.fit(load_X[train_indices])
transformed_X = sc.transform(load_X)

### Generate input sequences to a SHRED model (3D matrix of (total_time_steps - lags, lags, num_sensors))
all_data_in = np.zeros((n - lags, lags, num_sensors))
### for all total_time_steps - lags
for i in range(len(all_data_in)):
    ### for each time step, lags: look from i to 52 time steps into the future, sensor = sensor_locations
    all_data_in[i] = transformed_X[i:i+lags, sensor_locations]

### Generate training validation and test datasets both for reconstruction of states and forecasting sensors
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data_in = torch.tensor(all_data_in[train_indices], dtype=torch.float32).to(device)
valid_data_in = torch.tensor(all_data_in[valid_indices], dtype=torch.float32).to(device)
test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)

### -1 to have output be at the same time as final sensor measurements
### start of prediction is the same element as end of sensor measurement
train_data_out = torch.tensor(transformed_X[train_indices + lags - 1], dtype=torch.float32).to(device)
valid_data_out = torch.tensor(transformed_X[valid_indices + lags - 1], dtype=torch.float32).to(device)
test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

train_dataset = TimeSeriesDataset(train_data_in, train_data_out)
valid_dataset = TimeSeriesDataset(valid_data_in, valid_data_out)
test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

ValueError: ignored

In [ ]:
shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)
validation_errors = models.fit(shred, train_dataset, valid_dataset, batch_size=64, num_epochs=1000, lr=1e-3, verbose=True, patience=5)

In [ ]:
test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())
print(np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth))